<a href="https://colab.research.google.com/github/drfperez/openair/blob/main/Tropomi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# Cel·la de bash (executa amb !)
!sudo apt-get install r-base libudunits2-dev libgdal-dev libgeos-dev libproj-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libudunits2-dev is already the newest version (2.2.28-3).
r-base is already the newest version (4.4.2-1.2204.0).
libgdal-dev is already the newest version (3.6.4+dfsg-1~jammy0).
libgeos-dev is already the newest version (3.11.1-1~jammy0).
libproj-dev is already the newest version (9.1.1-1~jammy0).
0 upgraded, 0 newly installed, 0 to remove and 23 not upgraded.


In [ ]:

# Cel·la de R
install.packages("remotes")
remotes::install_github("r-spatial/rgee")
install.packages(c("terra", "sf", "ggplot2", "viridis"))

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)




proxy        (NA -> 0.4-27) [CRAN]
wk           (NA -> 0.9.4 ) [CRAN]
e1071        (NA -> 1.7-16) [CRAN]
units        (NA -> 0.8-5 ) [CRAN]
s2           (NA -> 1.1.7 ) [CRAN]
classInt     (NA -> 0.4-11) [CRAN]
terra        (NA -> 1.8-29) [CRAN]
sp           (NA -> 2.2-0 ) [CRAN]
lazyeval     (NA -> 0.2.2 ) [CRAN]
raster       (NA -> 3.6-31) [CRAN]
png          (NA -> 0.1-8 ) [CRAN]
leaflet.p... (NA -> 2.0.0 ) [CRAN]
crosstalk    (NA -> 1.2.1 ) [CRAN]
sfheaders    (NA -> 0.4.4 ) [CRAN]
rapidjsonr   (NA -> 1.2.0 ) [CRAN]
jsonify      (NA -> 1.2.2 ) [CRAN]
geometries   (NA -> 0.2.4 ) [CRAN]
sf           (NA -> 1.0-19) [CRAN]
leaflet      (NA -> 2.2.2 ) [CRAN]
geojsonsf    (NA -> 2.0.3 ) [CRAN]
here         (NA -> 1.0.1 ) [CRAN]
RcppTOML     (NA -> 0.2.2 ) [CRAN]
leafem       (NA -> 0.2.3 ) [CRAN]
reticulate   (NA -> 1.41.0) [CRAN]


Installing 24 packages: proxy, wk, e1071, units, s2, classInt, terra, sp, lazyeval, raster, png, leaflet.providers, crosstalk, sfheaders, rapidjsonr, jsonify, geometries, sf, leaflet, geojsonsf, here, RcppTOML, leafem, reticulate

Installing packages into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:

# Cel·la de R
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)  # Segueix l'enllaç per autenticar-te amb Google

# Definir àrea d'interès (Catalunya)
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# Carregar dades TROPOMI NO₂ (nivell 3, mitjana diària)
no2_collection <- ee$ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")$
  filterDate("2023-06-01", "2023-06-10")$  # Interval de dates
  select("NO2_column_number_density")$
  mean()$
  clip(catalunya)

# Processar dades
no2_proj <- no2_collection$reproject(crs = "EPSG:4326", scale = 1000)
no2_raster <- ee_as_raster(no2_proj, region = catalunya, via = "drive")

# Convertir a objecte "terra" i ajustar valors
no2_terra <- rast(no2_raster)
no2_terra <- no2_terra * 1e6  # Convertir a µmol/m²

# Visualització amb ggplot2
ggplot() +
  geom_spatraster(data = no2_terra) +
  geom_sf(data = catalunya, fill = NA, color = "white") +
  scale_fill_viridis(
    option = "inferno",
    na.value = NA,
    name = "NO₂ (µmol/m²)",
    limits = c(0, 200)  # Ajustar segons les dades
  ) +
  labs(
    title = "Concentració de NO₂ a Catalunya (TROPOMI, Juny 2023)",
    subtitle = "Dades: Sentinel-5P/TROPOMI",
    caption = "@Autor"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right"
  )

# Guardar resultat (opcional)
# ggsave("no2_catalunya.png", dpi = 300)

In [ ]:

# Carregar llibreries necessàries
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)

# Definir àrea d'interès (Catalunya)
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# --- SELECCIÓ DE PARÀMETRES ---
data_inici <- "2023-06-01"
data_fi <- "2023-06-10"
contaminant <- "NO2_column_number_density"  # Opcions: NO₂, SO₂, CO, etc.

# Seleccionar la col·lecció de dades segons el contaminant
s5p_collection <- ee$ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2")$  # Ajustar per altres contaminants si cal
  filterDate(data_inici, data_fi)$
  select(contaminant)$
  mean()$
  clip(catalunya)

# Processar dades
s5p_proj <- s5p_collection$reproject(crs = "EPSG:4326", scale = 1000)
s5p_raster <- ee_as_raster(s5p_proj, region = catalunya, via = "drive")

# Convertir a objecte "terra" i ajustar valors
s5p_terra <- rast(s5p_raster)
s5p_terra <- s5p_terra * 1e6  # Convertir a µmol/m² (ajust segons el contaminant)

# Visualització amb ggplot2
ggplot() +
  geom_spatraster(data = s5p_terra) +
  geom_sf(data = catalunya, fill = NA, color = "white") +
  scale_fill_viridis(
    option = "inferno",
    na.value = NA,
    name = paste0(contaminant, " (µmol/m²)"),
    limits = c(0, 200)  # Ajustar segons les dades
  ) +
  labs(
    title = paste("Concentració de", contaminant, "a Catalunya"),
    subtitle = paste("Període:", data_inici, "-", data_fi, "(TROPOMI)"),
    caption = "@Autor"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right"
  )

# Guardar resultat (opcional)
# ggsave(paste0("contaminacio_", contaminant, ".png"), dpi = 300)

In [ ]:

# Carregar llibreries
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)
library(glue)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)

# --- SELECCIÓ DE PARÀMETRES ---
data_inici <- "2023-06-01"
data_fi <- "2023-06-10"
contaminant <- "NO2"  # Opcions: "NO2", "SO2", "CO", "O3", "CH4"
resolucio <- 1000  # Resolució en metres

# --- DEFINIR ÀREA D'INTERÈS (Catalunya) ---
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# --- SELECCIONAR DADES SEGONS CONTAMINANT ---
s5p_collections <- list(
  "NO2" = list(id = "COPERNICUS/S5P/OFFL/L3_NO2", band = "NO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "SO2" = list(id = "COPERNICUS/S5P/OFFL/L3_SO2", band = "SO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "CO"  = list(id = "COPERNICUS/S5P/OFFL/L3_CO",  band = "CO_column_number_density",  escala = 1e2, unitat = "mol/m²"),
  "O3"  = list(id = "COPERNICUS/S5P/OFFL/L3_O3",  band = "O3_column_number_density",  escala = 1e3, unitat = "mmol/m²"),
  "CH4" = list(id = "COPERNICUS/S5P/OFFL/L3_CH4", band = "CH4_column_volume_mixing_ratio_dry_air", escala = 1e9, unitat = "ppb")
)

# Verificar si el contaminant és vàlid
if (!(contaminant %in% names(s5p_collections))) {
  stop(glue("ERROR: El contaminant '{contaminant}' no és vàlid. Tria entre: {paste(names(s5p_collections), collapse=', ')}."))
}

# Seleccionar la col·lecció corresponent
s5p_data <- s5p_collections[[contaminant]]
s5p_collection <- ee$ImageCollection(s5p_data$id)$
  filterDate(data_inici, data_fi)$
  select(s5p_data$band)$
  mean()$
  clip(catalunya)

# Processar dades
s5p_proj <- s5p_collection$reproject(crs = "EPSG:4326", scale = resolucio)
s5p_raster <- ee_as_raster(s5p_proj, region = catalunya, via = "drive")

# Convertir a objecte "terra" i ajustar valors segons escala
s5p_terra <- rast(s5p_raster) * s5p_data$escala

# --- VISUALITZACIÓ AMB ggplot2 ---
ggplot() +
  geom_spatraster(data = s5p_terra) +
  geom_sf(data = catalunya, fill = NA, color = "white") +
  scale_fill_viridis(
    option = "inferno",
    na.value = NA,
    name = glue("{contaminant} ({s5p_data$unitat})")
  ) +
  labs(
    title = glue("Concentració de {contaminant} a Catalunya ({data_inici} - {data_fi})"),
    subtitle = "Dades: Sentinel-5P/TROPOMI",
    caption = "@Autor"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right"
  )

# Guardar resultat (opcional)
# ggsave(glue("contaminacio_{contaminant}.png"), dpi = 300)

In [ ]:

# Carregar llibreries
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)
library(glue)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)

# --- Entrada d'usuari ---
cat("Introdueix la data d'inici (YYYY-MM-DD): ")
data_inici <- readline()
cat("Introdueix la data de fi (YYYY-MM-DD): ")
data_fi <- readline()

# Opcions disponibles
opcions_contaminants <- c("NO2", "SO2", "CO", "O3", "CH4")
cat("Selecciona un contaminant (NO2, SO2, CO, O3, CH4): ")
contaminant <- readline()

# Verificar si el contaminant és vàlid
if (!(contaminant %in% opcions_contaminants)) {
  stop(glue("ERROR: '{contaminant}' no és vàlid. Tria entre: {paste(opcions_contaminants, collapse=', ')}."))
}

# --- Definir l'àrea d'interès (Catalunya) ---
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# --- Seleccionar col·lecció de dades ---
s5p_collections <- list(
  "NO2" = list(id = "COPERNICUS/S5P/OFFL/L3_NO2", band = "NO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "SO2" = list(id = "COPERNICUS/S5P/OFFL/L3_SO2", band = "SO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "CO"  = list(id = "COPERNICUS/S5P/OFFL/L3_CO",  band = "CO_column_number_density",  escala = 1e2, unitat = "mol/m²"),
  "O3"  = list(id = "COPERNICUS/S5P/OFFL/L3_O3",  band = "O3_column_number_density",  escala = 1e3, unitat = "mmol/m²"),
  "CH4" = list(id = "COPERNICUS/S5P/OFFL/L3_CH4", band = "CH4_column_volume_mixing_ratio_dry_air", escala = 1e9, unitat = "ppb")
)

# Seleccionar col·lecció de dades
s5p_data <- s5p_collections[[contaminant]]
s5p_collection <- ee$ImageCollection(s5p_data$id)$
  filterDate(data_inici, data_fi)$
  select(s5p_data$band)$
  mean()$
  clip(catalunya)

# Processar dades
s5p_proj <- s5p_collection$reproject(crs = "EPSG:4326", scale = 1000)
s5p_raster <- ee_as_raster(s5p_proj, region = catalunya, via = "drive")

# Convertir a objecte "terra" i ajustar segons escala
s5p_terra <- rast(s5p_raster) * s5p_data$escala

# --- Visualització amb ggplot2 ---
ggplot() +
  geom_spatraster(data = s5p_terra) +
  geom_sf(data = catalunya, fill = NA, color = "white") +
  scale_fill_viridis(
    option = "inferno",
    na.value = NA,
    name = glue("{contaminant} ({s5p_data$unitat})")
  ) +
  labs(
    title = glue("Concentració de {contaminant} a Catalunya ({data_inici} - {data_fi})"),
    subtitle = "Dades: Sentinel-5P/TROPOMI",
    caption = "@Autor"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(face = "bold", size = 14),
    legend.position = "right"
  )

# Guardar resultat (opcional)
# ggsave(glue("contaminacio_{contaminant}.png"), dpi = 300)

In [ ]:

# Carregar llibreries
library(shiny)
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)
library(glue)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)

# --- Definir l'àrea d'interès (Catalunya) ---
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# --- Opcions disponibles ---
s5p_collections <- list(
  "NO2" = list(id = "COPERNICUS/S5P/OFFL/L3_NO2", band = "NO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "SO2" = list(id = "COPERNICUS/S5P/OFFL/L3_SO2", band = "SO2_column_number_density", escala = 1e6, unitat = "µmol/m²"),
  "CO"  = list(id = "COPERNICUS/S5P/OFFL/L3_CO",  band = "CO_column_number_density",  escala = 1e2, unitat = "mol/m²"),
  "O3"  = list(id = "COPERNICUS/S5P/OFFL/L3_O3",  band = "O3_column_number_density",  escala = 1e3, unitat = "mmol/m²"),
  "CH4" = list(id = "COPERNICUS/S5P/OFFL/L3_CH4", band = "CH4_column_volume_mixing_ratio_dry_air", escala = 1e9, unitat = "ppb")
)

# --- INTERFÍCIE `shiny` ---
ui <- fluidPage(
  titlePanel("Mapa de Contaminació a Catalunya"),

  sidebarLayout(
    sidebarPanel(
      dateInput("data_inici", "Data d'inici:", value = "2023-06-01"),
      dateInput("data_fi", "Data de fi:", value = "2023-06-10"),
      selectInput("contaminant", "Selecciona el contaminant:", choices = names(s5p_collections)),
      actionButton("generar", "Generar Mapa")
    ),

    mainPanel(
      plotOutput("mapa")
    )
  )
)

# --- SERVIDOR `shiny` ---
server <- function(input, output) {

  # Funció reactiva per obtenir les dades quan es prem el botó
  dades_contaminacio <- eventReactive(input$generar, {
    contaminant <- input$contaminant
    data_inici <- as.character(input$data_inici)
    data_fi <- as.character(input$data_fi)

    # Seleccionar col·lecció de dades
    s5p_data <- s5p_collections[[contaminant]]
    s5p_collection <- ee$ImageCollection(s5p_data$id)$
      filterDate(data_inici, data_fi)$
      select(s5p_data$band)$
      mean()$
      clip(catalunya)

    # Processar dades
    s5p_proj <- s5p_collection$reproject(crs = "EPSG:4326", scale = 1000)
    s5p_raster <- ee_as_raster(s5p_proj, region = catalunya, via = "drive")

    # Convertir a objecte "terra" i ajustar segons escala
    s5p_terra <- rast(s5p_raster) * s5p_data$escala
    list(terra = s5p_terra, unitat = s5p_data$unitat, contaminant = contaminant, data_inici = data_inici, data_fi = data_fi)
  })

  # Generar el mapa
  output$mapa <- renderPlot({
    req(dades_contaminacio())  # Espera fins que es generin les dades

    data <- dades_contaminacio()

    ggplot() +
      geom_spatraster(data = data$terra) +
      geom_sf(data = catalunya, fill = NA, color = "white") +
      scale_fill_viridis(
        option = "inferno",
        na.value = NA,
        name = glue("{data$contaminant} ({data$unitat})")
      ) +
      labs(
        title = glue("Concentració de {data$contaminant} a Catalunya"),
        subtitle = glue("{data$data_inici} - {data$data_fi} (Sentinel-5P)"),
        caption = "@Autor"
      ) +
      theme_minimal() +
      theme(
        plot.title = element_text(face = "bold", size = 14),
        legend.position = "right"
      )
  })
}

# Executar l'aplicació
shinyApp(ui = ui, server = server)

In [ ]:

# Carregar llibreries
library(shiny)
library(rgee)
library(terra)
library(sf)
library(ggplot2)
library(viridis)
library(glue)

# Inicialitzar Earth Engine
ee_Initialize(drive = TRUE)

# --- Definir l'àrea d'interès (Catalunya) ---
catalunya <- st_as_sfc(st_bbox(c(
  xmin = 0.15, ymin = 40.5, xmax = 3.3, ymax = 42.8
), crs = 4326)) %>%
  st_as_sf()

# --- Opcions disponibles (OMI) ---
omi_collections <- list(
  "NO2" = list(id = "NASA/OMI/NO2", band = "NO2", escala = 1e6, unitat = "µmol/m²"),
  "SO2" = list(id = "NASA/OMI/SO2", band = "SO2", escala = 1e6, unitat = "µmol/m²"),
  "O3"  = list(id = "NASA/OMI/O3",  band = "O3",  escala = 1e3, unitat = "mmol/m²"),
  "CO"  = list(id = "NASA/OMI/CO",  band = "CO",  escala = 1e2, unitat = "mol/m²")
)

# --- INTERFÍCIE `shiny` ---
ui <- fluidPage(
  titlePanel("Mapa de Contaminació a Catalunya (OMI)"),

  sidebarLayout(
    sidebarPanel(
      dateInput("data_inici", "Data d'inici:", value = "2023-06-01"),
      dateInput("data_fi", "Data de fi:", value = "2023-06-10"),
      selectInput("contaminant", "Selecciona el contaminant:", choices = names(omi_collections)),
      actionButton("generar", "Generar Mapa")
    ),

    mainPanel(
      plotOutput("mapa")
    )
  )
)

# --- SERVIDOR `shiny` ---
server <- function(input, output) {

  # Funció reactiva per obtenir les dades quan es prem el botó
  dades_contaminacio <- eventReactive(input$generar, {
    contaminant <- input$contaminant
    data_inici <- as.character(input$data_inici)
    data_fi <- as.character(input$data_fi)

    # Seleccionar col·lecció de dades OMI
    omi_data <- omi_collections[[contaminant]]
    omi_collection <- ee$ImageCollection(omi_data$id)$
      filterDate(data_inici, data_fi)$
      select(omi_data$band)$
      mean()$
      clip(catalunya)

    # Processar dades
    omi_proj <- omi_collection$reproject(crs = "EPSG:4326", scale = 1000)
    omi_raster <- ee_as_raster(omi_proj, region = catalunya, via = "drive")

    # Convertir a objecte "terra" i ajustar segons escala
    omi_terra <- rast(omi_raster) * omi_data$escala
    list(terra = omi_terra, unitat = omi_data$unitat, contaminant = contaminant, data_inici = data_inici, data_fi = data_fi)
  })

  # Generar el mapa
  output$mapa <- renderPlot({
    req(dades_contaminacio())  # Espera fins que es generin les dades

    data <- dades_contaminacio()

    ggplot() +
      geom_spatraster(data = data$terra) +
      geom_sf(data = catalunya, fill = NA, color = "white") +
      scale_fill_viridis(
        option = "inferno",
        na.value = NA,
        name = glue("{data$contaminant} ({data$unitat})")
      ) +
      labs(
        title = glue("Concentració de {data$contaminant} a Catalunya"),
        subtitle = glue("{data$data_inici} - {data$data_fi} (OMI)"),
        caption = "@Autor"
      ) +
      theme_minimal() +
      theme(
        plot.title = element_text(face = "bold", size = 14),
        legend.position = "right"
      )
  })
}

# Executar l'aplicació
shinyApp(ui = ui, server = server)